# 데이터 불러오기

In [0]:
import pandas as pd
import numpy as np
data = pd.read_csv("C:/Users/user/kym/감성분석Project/대한항공/대한항공_Tone까지(최종분석데이터).csv").dropna(axis=0)

In [0]:
data

NameError: ignored

In [0]:
data=data.sample(frac=1)
data=data.reset_index(drop=True)

In [0]:
Train_doc = data.loc[:21600]
Test_doc = data.loc[21600:]

In [0]:
Train_doc

In [0]:
X_train_doc = Train_doc['Tokens'].str.split(',').values.tolist()
y_train_label_doc = Train_doc['Label_3'].values.tolist()

X_test_doc = Test_doc['Tokens'].str.split(',').values.tolist()
y_test_label_doc = Test_doc['Label_3'].values.tolist()

In [0]:
print(X_train_doc[0:10], sep=' ')

# CNN 모델

In [0]:
from collections import defaultdict
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import random

In [0]:

# 단어에 대한 idx 부여
def convert_token_to_idx(token_ls):
    for tokens in token_ls:
        yield [token2idx[token] for token in tokens]
    return

In [0]:
X_train_doc[0]

# 문서 최대 토큰 길이 구하기

In [0]:
from eunjeon import Mecab
from collections import Counter

In [0]:
max_len =[]
for i in range(len(X_train_doc)):
    max_len.append(len(X_train_doc[i]))


In [0]:
max(max_len)

In [0]:
count_len=Counter(max_len).most_common()  

In [0]:
a=count_len.sort(reverse=True)

In [0]:
count_len

# Padding

In [0]:
token2idx = defaultdict(lambda : len(token2idx)) # token과 index를 매칭시켜주는 딕셔너리
pad = token2idx['<PAD>']  # pytorch Variable로 변환하기 위해, 문장의 길이를 맞춰주기 위한 padding 

x_train = list(convert_token_to_idx(X_train_doc))
x_test = list(convert_token_to_idx(X_test_doc))

idx2token = {val : key for key,val in token2idx.items()}

In [0]:
print(x_train[0:10], sep=' ')

In [0]:
def add_padding(token_ls, max_len):
    for i, tokens in enumerate(token_ls):
        n_token = len(tokens)
        
        # 길이가 짧으면 padding을 추가
        if n_token < max_len:
            token_ls[i] += [pad] * (max_len - n_token) # 부족한 만큼 padding을 추가함
        
        # 길이가 길면, max_len에서 짜름
        elif n_token > max_len:
            token_ls[i] = tokens[:max_len]
    return token_ls

In [0]:
max_len = 200
x_train = add_padding(x_train, max_len)
x_test = add_padding(x_test, max_len)

In [0]:
' '.join([idx2token[x] for x in x_train[0]])

In [0]:
def convert_to_long_variable(w2i_ls):
    return Variable(torch.LongTensor(w2i_ls))

In [0]:
x_train = convert_to_long_variable(x_train)
x_test = convert_to_long_variable(x_test)

y_train = convert_to_long_variable(y_train_label_doc)
y_test = convert_to_long_variable(y_test_label_doc)

In [0]:
x_train[0]

In [0]:
class CNN_text(nn.Module):
    
    def __init__(self, n_words, embed_size, pad_index, hid_size, drop_rate, kernel_size_ls, num_filter, n_class):
        super(CNN_text, self).__init__()
        
        self.pad_index = pad_index              # 단어 embedding 과정에서 제외시킬 padding token
        self.embed_size = embed_size            # 임베딩 차원의 크기
        self.hid_size = hid_size                # 히든 레이어 갯수
        self.drop_rate = drop_rate              # 드롭아웃 비율
        self.num_filter = num_filter            # 필터의 갯수 
        self.kernel_size_ls = kernel_size_ls    # 각기 다른 필터 사이즈가 담긴 리스트
        self.num_kernel = len(kernel_size_ls)   # 필터 사이즈의 종류 수
        self.n_class = n_class                  # 카테고리 갯수
        
        self.embed = nn.Embedding(
            num_embeddings=n_words, 
            embedding_dim=embed_size,
            padding_idx=self.pad_index
        )
        
        
        # kernel size는 (n-gram, embed_size)이다.
        # 커널의 열(column)의 크기는 embed_size와 일치하므로, 단어 임베딩 벡터를 모두 커버한다.
        # 따라서, n의 row 크기를 갖는 커널은 한번에 n개의 단어를 커버하는 n-gram 커널이라고 볼 수 있다.
        self.convs = nn.ModuleList([nn.Conv2d(1, num_filter, (kernel_size, embed_size)) for kernel_size in kernel_size_ls])
        
        self.lin = nn.Sequential(
            nn.Linear(self.num_kernel*num_filter, hid_size), nn.ReLU(), nn.Dropout(drop_rate),
            nn.Linear(hid_size, n_class),
        )
        
    def forward(self, x):
        embed = self.embed(x) # batch_size x max_length x embed_size
        embed.unsqueeze_(1)       # batch_size, 1, max_length, embed_size : convolution을 위해 4D로 차원을 조절
        
        # convolution
        conved = [conv(embed).squeeze(3) for conv in self.convs] # [batch_size, num_filter, max_length -kernel_size +1]
        
        # max_pooling
        pooled = [F.max_pool1d(conv, (conv.size(2))).squeeze(2) for conv in conved] # [batch_size, num_kernel, num_filter]
            
        # dropout
        dropouted = [F.dropout(pool, self.drop_rate) for pool in pooled]
        
        # concatenate
        concated = torch.cat(dropouted, dim = 1) # [batch_size, num_kernel * num_filter]
        logit = self.lin(concated)
        
        return logit

In [0]:

params = {
    'n_words' : len(token2idx),        # 고유한 단어 토큰의 갯수
    'embed_size' : 30,                # 임베딩 차원의 크기
    'pad_index' : token2idx['<PAD>'],  # 패딩 토큰
    'hid_size' : 100,                  # 히든 레이어 갯수
    'drop_rate' : 0.5,                 # 드롭아웃 비율          (원문에서는 0.5를 사용)
    'kernel_size_ls' : [3,4,5],      # 커널 사이즈 리스트        (원문애서는 3,4,5를 사용)
    'num_filter' : 32,                 # 각 사이즈 별 커널 갯수 (원문에서는 100을 사용)
    'n_class' : 3,                  # 카테고리 갯수
}

In [0]:
model = CNN_text(**params)

In [0]:
model

# 모델저장 및 불러오기

In [0]:
savePath = "C:/Users/user/kym/감성분석Project/대한항공/CNN_model.pth"
torch.save(model.state_dict(), savePath)

In [0]:
# # load
new_model = CNN_text(**params)
new_model.load_state_dict(torch.load("C:/Users/user/kym/감성분석Project/대한항공/CNN_model.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
new_model

CNN_text(
  (embed): Embedding(2007, 30, padding_idx=0)
  (convs): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(3, 30), stride=(1, 1))
    (1): Conv2d(1, 32, kernel_size=(4, 30), stride=(1, 1))
    (2): Conv2d(1, 32, kernel_size=(5, 30), stride=(1, 1))
  )
  (lin): Sequential(
    (0): Linear(in_features=96, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=100, out_features=3, bias=True)
  )
)

In [0]:
len(list(new_model.parameters()))

11

# 모델 Training 및 Backtesting

In [0]:
epochs = 5
lr = 0.01
batch_size = 500

train_idx = np.arange(x_train.size(0))
test_idx = np.arange(x_test.size(0))
optimizer = torch.optim.Adam(new_model.parameters(),lr) # 원문에서는 Adadelta 알고리즘을 사용
criterion = nn.CrossEntropyLoss(reduction='sum')

loss_ls = []

for epoch in range(epochs):
    new_model.train()
    
    # input 데이터 순서 섞기
    random.shuffle(train_idx)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]
    train_loss = 0

    for start_idx, end_idx in zip(range(0, x_train.size(0), batch_size),
                                  range(batch_size, x_train.size(0)+1, batch_size)):
        
        x_batch = x_train[start_idx : end_idx]
        y_batch = y_train[start_idx : end_idx].long()
        
        scores = new_model(x_batch)
        predict = F.softmax(scores, dim=1).argmax(dim = 1)
        
        acc = (predict == y_batch).sum().item() / batch_size
        
        loss = criterion(scores, y_batch)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('Train epoch : %s,  loss : %s,  accuracy :%.3f'%(epoch+1, train_loss / batch_size, acc))
    print('=================================================================================================')
    
    loss_ls.append(train_loss)
    
    if (epoch+1) % 10 == 0:
        new_model.eval()
        scores = new_model(x_test)
        predict = F.softmax(scores, dim=1).argmax(dim = 1)
        
        acc = (predict == y_test.long()).sum().item() / y_test.size(0)
        loss = criterion(scores, y_test.long())
        
        print('*************************************************************************************************')
        print('*************************************************************************************************')
        print('Test Epoch : %s, Test Loss : %.03f , Test Accuracy : %.03f'%(epoch+1, loss.item()/y_test.size(0), acc))
        print('*************************************************************************************************')
        print('*************************************************************************************************')

Train epoch : 1,  loss : 34.64312365722656,  accuracy :0.610
Train epoch : 2,  loss : 32.08012243652344,  accuracy :0.602
Train epoch : 3,  loss : 31.980193298339845,  accuracy :0.612
Train epoch : 4,  loss : 31.704383850097656,  accuracy :0.636
Train epoch : 5,  loss : 31.39335986328125,  accuracy :0.624


# 특정 기간 예측

In [0]:
x_predict_data = X_train_doc[50]

In [0]:
data['Date'][50]

'2017-03-10'

In [0]:
data['Url'][50]

'http://www.asiatoday.co.kr/view.php?key=20170310010006761'

In [0]:
data['Content'][50]

'사측과의 임금협상을 매듭짓지 못한 대한항공 조종사 노동조합이 두 달여만에 다시 파업에 돌입한다.대항항공 조종사 노조는 일부터 일까지 일간 파업을 벌인다고 일 밝혔다. 대한항공 노사는 이날 오후 시께 년 임금협상과 관련해 차 교섭을 벌였지만 사측이 제시한 수정안을 노조가 수용하지 않으면서 파업이 결정됐다.사측은 년 . 년 총액 대비 .의 임금 인상안을 제시했다. 또 보안수당을 만원에서 만원으로 올려 지난해 월 일부터 적용하겠다는 뜻을 전달했다.노조는 이는 조합이 바라는 변화와는 간극이 너무나 크다며 앞으로도 교섭의 장을 닫지는 않을 것이나 변화를 찾기 위한 행동을 준비할 수밖에 없다고 밝혔다.앞서 노조는 지난해 월 일부터 파업을 벌였다가 임금교섭에 집중하겠다며 일주일 만인 월 일 잠정 중단한 바 있다. 당시 노조는 조종사 인력 유출을 막고 경영진만 수익을 누리는 잘못된 구조를 개선하기 위해 실질 임금 인상이 필요하다고 주장했다.노조는 차 파업에 참여했던 조합원을 제외하고 총 명의 차 파업 참가자를 무작위 선발해 파업을 진행할 계획이다.안소연 기자 ..대한항공 조종사 노조 파업 '

In [0]:
data['Rate'][50]

0.057657889000000004

In [0]:
data['AC'][50]

0.442758654

In [0]:
data['Label_3'][50]

2.0

In [0]:
# 토큰화 된 1개의 문서를 이중리스트로 만듦 
x_predict_data_2 = []
x_predict_data_2.append(x_predict_data)

In [0]:
x_predict_data_2

[['임금',
  '협상',
  '못한',
  '대한항공',
  '조종사',
  '노동조합',
  '다시',
  '파업',
  '돌입',
  '항공',
  '조종사',
  '노조',
  '일간',
  '파업',
  '밝혔',
  '대한항공',
  '노사',
  '이날',
  '임금',
  '협상',
  '관련',
  '제시',
  '노조',
  '수용',
  '파업',
  '결정',
  '총액',
  '대비',
  '임금',
  '제시',
  '보안',
  '지난해',
  '적용',
  '전달',
  '노조',
  '변화',
  '변화',
  '위한',
  '행동',
  '준비',
  '밝혔',
  '앞서',
  '노조',
  '지난해',
  '파업',
  '임금',
  '집중',
  '주일',
  '중단',
  '당시',
  '노조',
  '조종사',
  '인력',
  '경영진',
  '수익',
  '잘못',
  '구조',
  '개선',
  '위해',
  '실질',
  '임금',
  '인상',
  '필요',
  '주장',
  '노조',
  '파업',
  '참여',
  '제외',
  '파업',
  '선발',
  '파업',
  '계획',
  '안소연',
  '대한항공',
  '조종사',
  '노조',
  '파업']]

In [0]:
x_test = list(convert_token_to_idx(x_predict_data_2))

In [0]:
len(x_test)
# 1개의 문서

1

In [0]:
x_test = add_padding(x_test, max_len)

In [0]:
x_test = convert_to_long_variable(x_test)

In [0]:
x_test
# 임베딩 값

tensor([[1505, 1506,  816,   25,   47, 1507,   56, 1508, 1509,   27,   47, 1510,
          283, 1508,  162,   25, 1511,   95, 1505, 1506,   97, 1237, 1510,  935,
         1508,  287, 1233,  772, 1505, 1237, 1064,  109,   60,  120, 1510,  342,
          342,  334,  326, 1348,  162,  598, 1510,  109, 1508, 1505,  242, 1512,
          920,  921, 1510,   47,    6, 1292,  737, 1327,  125,  741,   40, 1402,
         1505,  668,   89, 1098, 1510, 1508,  874,  228, 1508,  259, 1508,  164,
         1513,   25,   47, 1510, 1508,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [0]:
scores = new_model(x_test)
predict = F.softmax(scores, dim=1).argmax(dim = 1)

In [0]:
scores

tensor([[ 1.0729, -1.7293,  0.7453]], grad_fn=<AddmmBackward>)

In [0]:
predict
# 예측 Label

tensor([0])

In [0]:
predict_ls = predict.numpy()

In [0]:
predict_ls[0]

0

In [0]:
F.softmax(scores, dim=1)
# 각각 0, 1, 2 값이 나올 확률

tensor([[0.5614, 0.0341, 0.4045]], grad_fn=<SoftmaxBackward>)